**Import Library**

In [501]:
import numpy as np
import matplotlib.pyplot as plt
import matplotlib.image as im
import sympy as sp
from PIL import Image, ImageOps
import os

**Function**

In [511]:
#Process the Image into a flat 1D array
def ProcessImage(img):
    Grascale = ImageOps.grayscale(img)
    Facematrix = im.pil_to_array(Grascale)
    FlatFaceArray = np.concat(Facematrix,axis=None)
    return FlatFaceArray


**Size of the data**

In [512]:
imgdir = "Data\\" + "1.jpg"
Face= Image.open(imgdir)
GrascaleFace=ImageOps.grayscale(Face)
FaceArray = im.pil_to_array(GrascaleFace)
Height, Width = np.shape(FaceArray)
num_files = sum(1 for entry in os.scandir("Data") if entry.is_file())

**Make the Data Matrix**

In [513]:
for i in range(num_files):
    imgdir = "Data\\" + str(i) + ".jpg"
#Nhap hinh anh
    Face = Image.open(imgdir)
#Process hinh anh
    FlatFaceArray = ProcessImage(Face)
    if i == 0:
        FaceMatrix = FlatFaceArray
    else:
        FaceMatrix = np.column_stack((FaceMatrix,FlatFaceArray))

In [514]:
MeanFace=np.mean(FaceMatrix, axis=1)
ColumnMeanFace= MeanFace[:, np.newaxis]
print(MeanFace)

[203.9375  203.9375  203.9375  ... 209.875   210.46875 211.15625]


In [ ]:
CenteredFaceMatrix=FaceMatrix  - ColumnMeanFace
#Find the Eigenface of the Data
BtB=CenteredFaceMatrix.T @ CenteredFaceMatrix
V, S, Vt = np.linalg.svd(BtB)
V
S_matrix=np.sqrt(np.diag(S))
S_inv=np.linalg.inv(S_matrix)
U=CenteredFaceMatrix @ V @ S_inv
U

array([[ 2.21428736e-06,  1.87218365e-05, -4.21816427e-07, ...,
         4.51699165e-05,  1.40074906e-05, -2.03875229e-12],
       [ 2.21428736e-06,  1.87218365e-05, -4.21816427e-07, ...,
         4.51699165e-05,  1.40074906e-05, -2.03875229e-12],
       [ 2.21428736e-06,  1.87218365e-05, -4.21816427e-07, ...,
         4.51699165e-05,  1.40074906e-05, -2.03875229e-12],
       ...,
       [ 7.04940586e-04,  1.80757914e-03, -2.09004523e-04, ...,
        -1.17098503e-02, -9.72644649e-03, -4.41702479e-09],
       [ 6.90363274e-04,  1.73995423e-03, -1.26370344e-04, ...,
        -1.09582842e-02, -8.97263676e-03, -4.16281956e-09],
       [ 7.00395487e-04,  1.63076503e-03,  1.09699753e-05, ...,
        -9.46982822e-03, -7.61477301e-03, -3.65220725e-09]])

In [ ]:
#Reconstruct the Eigenface as a picture
Eigenface=U @ S_matrix +ColumnMeanFace
face = Eigenface[:,1]
face = np.split(face, Height)
for i in range(len(face)):
    if i == 0:
        recface=face[0]
    else:
        recface = np.vstack((recface,face[i]))
recim = Image.fromarray(recface).convert("L")
recim.save('test.png','PNG')

In [507]:
#let G be the face that need to be recognize
G = np.random.rand(U[:,1].shape[0])
# the cotribution of a eigenface to it is calculated by
#Let Omega be the contribution vector

def Facespace(vector):
    Omega = np.array([])
    for i in range(num_files):
        diff=U[:,i] @ vector
        Omega = np.append(Omega,[[diff]] )
    return Omega
#Omega= Omega[:, np.newaxis]
#Find the threshhold for the picture to be identified as a face, let that be epsilon
epsilon=0
S = 0
for i in range(num_files):
    S+= Facespace(CenteredFaceMatrix[:,i])
    avgOmega = S/len(range(num_files))
for i in range(num_files):
    e = np.linalg.norm(Facespace(CenteredFaceMatrix[:,i])-avgOmega)
    if e > epsilon:
        epsilon = e

In [508]:

#Input = ProcessImage(Image.open("32.JPG"))-MeanFace
#Input.shape

image=Image.open("32.JPG")
Omega = Facespace(ProcessImage(image)-MeanFace)
epsilon > np.linalg.norm(Omega-avgOmega)


np.False_